In [1]:
import keras

Using TensorFlow backend.


In [2]:
print('keras: ', keras.__version__)

keras:  2.0.8


In [3]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import numpy as np
import os
import random
import tensorflow as tf

In [4]:
# Get images
image_raw = img_to_array(load_img('dog.jpg'))
image_raw = np.array(image_raw, dtype=float)
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)

In [5]:
X = rgb2lab(1.0/255*image_raw)[:,:,0]
Y = rgb2lab(1.0/255*image_raw)[:,:,1:]
Y /= 128
X = X.reshape(1, 256, 256, 1)
Y = Y.reshape(1, 256, 256, 2)

In [10]:
def conv_stack(data, filters, s):
        output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

#Create inception embedding
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
embed = inception.predict(img)
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) # shape: (None, 28*28, 256)
fusion_output = Permute((2, 1))(fusion_output) # shape: (None, 256, 28*28)
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)# shape: (None, 256, 28, 28)
fusion_output = concatenate([fusion_output, encoder_output], axis=3) # (None, 512, 28, 28)
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) # (None, 256, 28, 28) and Eq. (5)

#Decoder
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 128, 1)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

(?, 64, 64, 256)
(?, 64, 64, 64)
(?, 128, 128, 64)
(?, 128, 128, 32)
(?, 128, 128, 2)
(?, 256, 256, 2)


In [11]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit([X, embede], Y,
    batch_size=1,
    epochs=1000)

Epoch 1/1000
1/1 [==============================] - 3s - loss: 0.4416
Epoch 2/1000
1/1 [==============================] - 0s - loss: 0.2720
Epoch 3/1000
1/1 [==============================] - 0s - loss: 0.2223
Epoch 4/1000
1/1 [==============================] - 0s - loss: 0.1094
Epoch 5/1000
1/1 [==============================] - 0s - loss: 0.0973
Epoch 6/1000
1/1 [==============================] - 0s - loss: 0.1230
Epoch 7/1000
1/1 [==============================] - 0s - loss: 0.0644
Epoch 8/1000
1/1 [==============================] - 0s - loss: 0.0379
Epoch 9/1000
1/1 [==============================] - 0s - loss: 0.0314
Epoch 10/1000
1/1 [==============================] - 0s - loss: 0.0288
Epoch 11/1000
1/1 [==============================] - 0s - loss: 0.0257
Epoch 12/1000
1/1 [==============================] - 0s - loss: 0.0225
Epoch 13/1000
1/1 [==============================] - 0s - loss: 0.0211
Epoch 14/1000
1/1 [==============================] - 0s - loss: 0.0182
Epoch 15/1000
1

1/1 [==============================] - 0s - loss: 0.0033
Epoch 117/1000
1/1 [==============================] - 0s - loss: 0.0030
Epoch 118/1000
1/1 [==============================] - 0s - loss: 0.0036
Epoch 119/1000
1/1 [==============================] - 0s - loss: 0.0032
Epoch 120/1000
1/1 [==============================] - 0s - loss: 0.0036
Epoch 121/1000
1/1 [==============================] - 0s - loss: 0.0026
Epoch 122/1000
1/1 [==============================] - 0s - loss: 0.0030
Epoch 123/1000
1/1 [==============================] - 0s - loss: 0.0025
Epoch 124/1000
1/1 [==============================] - 0s - loss: 0.0025
Epoch 125/1000
1/1 [==============================] - 0s - loss: 0.0033
Epoch 126/1000
1/1 [==============================] - 0s - loss: 0.0037
Epoch 127/1000
1/1 [==============================] - 0s - loss: 0.0028
Epoch 128/1000
1/1 [==============================] - 0s - loss: 0.0029
Epoch 129/1000
1/1 [==============================] - 0s - loss: 0.0041
Epoch 1

1/1 [==============================] - 0s - loss: 0.0020
Epoch 231/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 232/1000
1/1 [==============================] - 0s - loss: 0.0019
Epoch 233/1000
1/1 [==============================] - 0s - loss: 0.0019
Epoch 234/1000
1/1 [==============================] - 0s - loss: 0.0022
Epoch 235/1000
1/1 [==============================] - 0s - loss: 0.0020
Epoch 236/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 237/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 238/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 239/1000
1/1 [==============================] - 0s - loss: 0.0020
Epoch 240/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 241/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 242/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 243/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 2

1/1 [==============================] - 0s - loss: 0.0011
Epoch 345/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 346/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 347/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 348/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 349/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 350/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 351/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 352/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 353/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 354/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 355/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 356/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 357/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 3

1/1 [==============================] - 0s - loss: 0.0012
Epoch 458/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 459/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 460/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 461/1000
1/1 [==============================] - 0s - loss: 9.7835e-04
Epoch 462/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 463/1000
1/1 [==============================] - 0s - loss: 9.3825e-04
Epoch 464/1000
1/1 [==============================] - 0s - loss: 9.8138e-04
Epoch 465/1000
1/1 [==============================] - 0s - loss: 9.7010e-04
Epoch 466/1000
1/1 [==============================] - 0s - loss: 9.5380e-04
Epoch 467/1000
1/1 [==============================] - 0s - loss: 9.9716e-04
Epoch 468/1000
1/1 [==============================] - 0s - loss: 9.9755e-04
Epoch 469/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 470/1000
1/1 [==============================]

1/1 [==============================] - 0s - loss: 9.6026e-04
Epoch 570/1000
1/1 [==============================] - 0s - loss: 9.4505e-04
Epoch 571/1000
1/1 [==============================] - 0s - loss: 9.2951e-04
Epoch 572/1000
1/1 [==============================] - 0s - loss: 9.7105e-04
Epoch 573/1000
1/1 [==============================] - 0s - loss: 9.8425e-04
Epoch 574/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 575/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 576/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 577/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 578/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 579/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 580/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 581/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 582/1000
1/1 [==============================] - 0s - 

1/1 [==============================] - 0s - loss: 0.0010
Epoch 680/1000
1/1 [==============================] - 0s - loss: 9.7651e-04
Epoch 681/1000
1/1 [==============================] - 0s - loss: 9.1618e-04
Epoch 682/1000
1/1 [==============================] - 0s - loss: 8.7831e-04
Epoch 683/1000
1/1 [==============================] - 0s - loss: 8.8525e-04
Epoch 684/1000
1/1 [==============================] - 0s - loss: 9.0834e-04
Epoch 685/1000
1/1 [==============================] - 0s - loss: 9.4554e-04
Epoch 686/1000
1/1 [==============================] - 0s - loss: 9.4190e-04
Epoch 687/1000
1/1 [==============================] - 0s - loss: 9.3774e-04
Epoch 688/1000
1/1 [==============================] - 0s - loss: 9.3821e-04
Epoch 689/1000
1/1 [==============================] - 0s - loss: 9.6630e-04
Epoch 690/1000
1/1 [==============================] - 0s - loss: 9.5667e-04
Epoch 691/1000
1/1 [==============================] - 0s - loss: 9.8753e-04
Epoch 692/1000
1/1 [===========

1/1 [==============================] - 0s - loss: 0.0010
Epoch 789/1000
1/1 [==============================] - 0s - loss: 9.6098e-04
Epoch 790/1000
1/1 [==============================] - 0s - loss: 9.0131e-04
Epoch 791/1000
1/1 [==============================] - 0s - loss: 9.4958e-04
Epoch 792/1000
1/1 [==============================] - 0s - loss: 9.5173e-04
Epoch 793/1000
1/1 [==============================] - 0s - loss: 9.0902e-04
Epoch 794/1000
1/1 [==============================] - 0s - loss: 9.6623e-04
Epoch 795/1000
1/1 [==============================] - 0s - loss: 9.4403e-04
Epoch 796/1000
1/1 [==============================] - 0s - loss: 9.6146e-04
Epoch 797/1000
1/1 [==============================] - 0s - loss: 9.8191e-04
Epoch 798/1000
1/1 [==============================] - 0s - loss: 9.4843e-04
Epoch 799/1000
1/1 [==============================] - 0s - loss: 9.0349e-04
Epoch 800/1000
1/1 [==============================] - 0s - loss: 8.8102e-04
Epoch 801/1000
1/1 [===========

1/1 [==============================] - 0s - loss: 9.0147e-04
Epoch 897/1000
1/1 [==============================] - 0s - loss: 8.9593e-04
Epoch 898/1000
1/1 [==============================] - 0s - loss: 8.9142e-04
Epoch 899/1000
1/1 [==============================] - 0s - loss: 9.1055e-04
Epoch 900/1000
1/1 [==============================] - 0s - loss: 8.8525e-04
Epoch 901/1000
1/1 [==============================] - 0s - loss: 8.7824e-04
Epoch 902/1000
1/1 [==============================] - 0s - loss: 9.0732e-04
Epoch 903/1000
1/1 [==============================] - 0s - loss: 9.3886e-04
Epoch 904/1000
1/1 [==============================] - 0s - loss: 9.5525e-04
Epoch 905/1000
1/1 [==============================] - 0s - loss: 9.1766e-04
Epoch 906/1000
1/1 [==============================] - 0s - loss: 8.7640e-04
Epoch 907/1000
1/1 [==============================] - 0s - loss: 9.2686e-04
Epoch 908/1000
1/1 [==============================] - 0s - loss: 8.9036e-04
Epoch 909/1000
1/1 [=======

In [15]:
output = model.predict([X, embed])
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 53 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
#embed_output = UpSampling2D((2, 2))(embed_input)
#embed_output = UpSampling2D((2, 2))(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_output)
# embed_output = Dense(1024, activation='relu')(embed_output)
# embed_output = Dense(512, activation='relu')(embed_output)
# embed_output = Dense(256, activation='relu')(embed_output)
# embed_output = Reshape(([256,]))(embed_output)